# Segmenting and Clustering Neighborhoods in Toronto

### Hassan Ashiq 
- #### Connect with me on Linkdin : https://www.linkedin.com/in/hassan-ashiq

# *Part 1*

## Step 1 : Data Gathering from Wikipedia

In [218]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
from sklearn.cluster import KMeans

In [99]:
page=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup=BeautifulSoup(page.content, 'html.parser')


In [100]:
table_class='mw-parser-output'
items=soup.find(class_='mw-parser-output')
table_class=items.find(class_='wikitable sortable')
required_data=tabled.find_all('tr')

#### Creating DataFrame

In [137]:
#column_names = ['Postal-code','Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame()

In [138]:
for i in range(1,len(required_data)):
    td=required_data[i].find_all('td')
    postal_code=td[0].get_text().strip()
    borough=td[1].get_text().strip()
    neighbor=td[2].get_text().strip()
    
    neighborhoods=neighborhoods.append({'Postel-code':postal_code,
                         'Borough':borough,
                         'Neighborhood':neighbor}, ignore_index=True)

In [139]:
neighborhoods=neighborhoods[['Postel-code', 'Borough','Neighborhood']]

In [140]:
neighborhoods.shape

(180, 3)

## Step 2 Data Cleanup

#### 2.1 : removing rows with Not Assigned Borough

In [141]:
neighborhoods.drop(neighborhoods[neighborhoods.Borough=='Not assigned'].index, inplace=True)


#### 2.2 : More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [142]:
neighborhoods["Neighborhood"] =neighborhoods.groupby("Postel-code")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))

#### 2.3 : dropping duplicates

In [144]:
neighborhoods=neighborhoods.drop_duplicates()

#### 2.4 : If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [147]:
neighborhoods['Neighborhood'].replace("Not assigned", neighborhoods["Borough"],inplace=True)

In [154]:
#updating the index 
neighborhoods=neighborhoods.reset_index(drop=True)

In [155]:
neighborhoods

,Postel-code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Step 3 : Data Frame Shape Output

In [156]:
neighborhoods.shape

(103, 3)

# *Part 2*

### Step 4.a : Reading Geospatial Coordinates

In [168]:
coordinates_file=pd.read_csv('Geospatial_Coordinates.csv')
coordinates_file.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


*We will merge the two dataframes. To get the correct result independant on the order of data, we need to set indexes of two dataframes to its Postal Code columns*

In [170]:
neighborhoods.set_index('Postel-code', inplace=True)
coordinates_file.set_index('Postal Code', inplace=True)

*We will join the two files with respect to its Postal Codes*

In [175]:
toronto_df = pd.concat([neighborhoods, coordinates_file], axis=1, join='inner')

In [179]:
toronto_df.index.name="Postel-Code"

In [182]:
toronto_df.reset_index(inplace=True)

### Step 4.b : Output of DataFrame

In [183]:
toronto_df

,Postel-Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# *Part 3* : Explore and cluster the neighborhoods in Toronto

In [185]:
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [188]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tl-toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


#### Please Note :
*You many not see the Folium Map in Github because GitHub doesnot support it* 

*You can view it in https://nbviewer.jupyter.org/*

*just put this notebook link there.*

#### Using Folium to create a Map of Toronto with Boroughs markers on top

In [196]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Postel-Code'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

*Reduce the number of Boroughs to explore To reduce the numbers of calls to FourSquare API, we will *only explore boroughs that have Toronto in their names.*

In [193]:
toronto_boroughs = ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']
toronto_central_df = toronto_df[toronto_df['Borough'].isin(toronto_boroughs)].reset_index(drop=True)
print(toronto_central_df.shape)
toronto_central_df.head()

(39, 5)


,Postel-Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [197]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, long, post, borough, neigh in zip(toronto_central_df['Latitude'], toronto_central_df['Longitude'], toronto_central_df['Postel-Code'], toronto_central_df['Borough'], toronto_central_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

#### *Using FourSquare API to explore the Boroughs*

In [203]:
CLIENT_ID = 'MQQXHBN1NLGAYJG445WR5EWZ0OSI4SP2YKWMHSU53AQB3MF2' # your Foursquare ID
CLIENT_SECRET = 'GKQC1M3LQFGRCGBHWQABJA1CG0CLPP0FDK1UG4X4YFZV2ANK' # your Foursquare Secret
VERSION = '20190330' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MQQXHBN1NLGAYJG445WR5EWZ0OSI4SP2YKWMHSU53AQB3MF2
CLIENT_SECRET:GKQC1M3LQFGRCGBHWQABJA1CG0CLPP0FDK1UG4X4YFZV2ANK


In [204]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_central_df['Latitude'], toronto_central_df['Longitude'], toronto_central_df['Postel-Code'], toronto_central_df['Borough'], toronto_central_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [222]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['Postel-Code', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(1620, 9)


,Postel-Code,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


In [223]:
venues_df.groupby(['Postel-Code', 'Borough', 'Neighborhood'])['VenueName'].count()

Postel-Code  Borough           Neighborhood                                                                                              
M4E          East Toronto      The Beaches                                                                                                     5
M4K          East Toronto      The Danforth West, Riverdale                                                                                   43
M4L          East Toronto      India Bazaar, The Beaches West                                                                                 24
M4M          East Toronto      Studio District                                                                                                40
M4N          Central Toronto   Lawrence Park                                                                                                   4
M4P          Central Toronto   Davisville North                                                                                          

In [224]:
len(venues_df['VenueCategory'].unique())

237

so there are 237 unique venues

#### Analyze venues in each area

In [225]:
# one hot encoding
toronto_central_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_central_onehot['Postel-Code'] = venues_df['Postel-Code'] 
toronto_central_onehot['Borough'] = venues_df['Borough'] 
toronto_central_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_central_onehot.columns[-3:]) + list(toronto_central_onehot.columns[:-3])
toronto_central_onehot = toronto_central_onehot[fixed_columns]

print(toronto_central_onehot.shape)
toronto_central_onehot.head()

(1620, 240)


,Postel-Code,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Get the frequency of occurance of each category in an area

In [226]:
toronto_central_venues_freq = toronto_central_onehot.groupby(['Postel-Code', 'Borough', 'Neighborhoods']).mean().reset_index()
print(toronto_central_venues_freq.shape)
toronto_central_venues_freq.head()

(39, 240)


,Postel-Code,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.200000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.023256,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.023256
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000000
3,M4M,East Toronto,Studio District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.025,0.0,0.0,0.025000
4,M4N,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000000


#### Get 10 most occurance venue types in each area

In [228]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['Postel-Code', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postel-Code'] = toronto_central_venues_freq['Postel-Code']
neighborhoods_venues_sorted['Borough'] = toronto_central_venues_freq['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_central_venues_freq['Neighborhoods']

for ind in np.arange(toronto_central_venues_freq.shape[0]):
    row_categories = toronto_central_venues_freq.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
neighborhoods_venues_sorted

,Postel-Code,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Boutique,Boat or Ferry,Rental Car Location
31,M6H,West Toronto,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Pet Store,Supermarket,Café,Bar,Recording Studio,Bank,Middle Eastern Restaurant,Park
32,M6J,West Toronto,"Little Portugal, Trinity",Bar,Café,Coffee Shop,Asian Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Restaurant,Men's Store,Japanese Restaurant,Juice Bar
33,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Coffee Shop,Breakfast Spot,Yoga Studio,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Neighborhood,Sandwich Place,Cheese Shop,Pet Store
15,M5C,Downtown Toronto,St. James Town,Café,Coffee Shop,Gastropub,Restaurant,American Restaurant,Cocktail Bar,Lingerie Store,Moroccan Restaurant,Creperie,Hotel
25,M5S,Downtown Toronto,"University of Toronto, Harbord",Café,Japanese Restaurant,Bar,Italian Restaurant,Bookstore,Restaurant,Bakery,Chinese Restaurant,Pub,Sushi Restaurant
34,M6P,West Toronto,"High Park, The Junction South",Café,Mexican Restaurant,Furniture / Home Store,Thai Restaurant,Restaurant,Speakeasy,Flea Market,Bar,Bakery,Fast Food Restaurant
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,Pizza Place,Pub,Middle Eastern Restaurant,BBQ Joint,Indian Restaurant,History Museum,Burger Joint
26,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",Café,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Dessert Shop,Park,Grocery Store,Pizza Place


#### Clustering areas

In [230]:

kclusters = 4

toronto_central_venues_freq_clustering = toronto_central_venues_freq.drop(['Postel-Code', 'Borough', 'Neighborhoods'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_central_venues_freq_clustering)

toronto_central_clustered_df = toronto_central_df

toronto_central_clustered_df['Cluster'] = kmeans.labels_

toronto_central_clustered_df = toronto_central_clustered_df.join(neighborhoods_venues_sorted.drop(['Borough', 'Neighborhoods'], 1).set_index('Postel-Code'), on='Postel-Code')
toronto_central_clustered_df.sort_values(['Cluster'] + freqColumns, inplace=True)
toronto_central_clustered_df


,Postel-Code,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Fast Food Restaurant,Rental Car Location,Mexican Restaurant,Shoe Store,Restaurant,Café,Sporting Goods Shop
10,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,0,Coffee Shop,Aquarium,Café,Hotel,Scenic Lookout,Brewery,Sporting Goods Shop,Restaurant,Fried Chicken Joint,Bar
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Gym,Thai Restaurant,Hotel,Deli / Bodega,Sushi Restaurant,Bookstore,Pizza Place
32,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,1,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Boutique,Boat or Ferry,Rental Car Location
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1,Bakery,Pharmacy,Pet Store,Supermarket,Café,Bar,Recording Studio,Bank,Middle Eastern Restaurant,Park
11,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,1,Bar,Café,Coffee Shop,Asian Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Restaurant,Men's Store,Japanese Restaurant,Juice Bar
14,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,1,Café,Bakery,Coffee Shop,Breakfast Spot,Yoga Studio,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant
17,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Neighborhood,Sandwich Place,Cheese Shop,Pet Store
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Gastropub,Restaurant,American Restaurant,Cocktail Bar,Lingerie Store,Moroccan Restaurant,Creperie,Hotel
27,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,1,Café,Japanese Restaurant,Bar,Italian Restaurant,Bookstore,Restaurant,Bakery,Chinese Restaurant,Pub,Sushi Restaurant


#### Show those clusters onto a map


In [231]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_central_clustered_df['Latitude'], toronto_central_clustered_df['Longitude'], toronto_central_clustered_df['Postel-Code'], toronto_central_clustered_df['Borough'], toronto_central_clustered_df['Neighborhood'], toronto_central_clustered_df['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


**Upon observing the result, we can name the clusters as follow:**

- *Cluster 0 (Red): Living area (with mostly park, trail, school; and some small businesses)*

- *Cluster 1 (Yellow): Roselawn - Central Toronto (nothing here except a garden)*

- *Cluster 2 (Purple): Business area (with lots of business venues)*

- *Cluster 3 (Blue) : Oldest neighboorhood with lot of nature park*

**Thank for following my project.**

### Hassan Ashiq 
- #### Connect with me on Linkdin : https://www.linkedin.com/in/hassan-ashiq